# Binary classification

What follows is the code implementing the linear classification example.

It does not converge -- and neither does any other unmodified implementation I have found on the web.

In [ ]:
from random import choice
from utils.sugar import *

dataset = (((1.2, 0.7), +1.0), ((-0.3, 0.5), -1.0), ((-3.0, -1.0), +1.0),
           ((0.1, 1.0), -1.0), ((3.0, 1.1), -1.0), ((2.1, -3.0), +1.0))

a, b, c = param(1, -2, -1)  # initial solution
x, y, label = const(0, 0, 0)  # not affected by backprop
f = minimum(1, label * (a * x + b * y + c))
regularization = 1.0

for iteration in range(500):

    if iteration % 50 == 0 or (iteration % 10 == 0 and iteration < 40):
        correct = sum(f.compute() > 0 for (x.val, y.val), label.val in dataset)
        print 'Accuracy at iteration {}: {:.1f} [{:.2f} {:.2f} {:.2f}]'.format(
            iteration, (100.0 * correct) / len(dataset), a.val, b.val, c.val)

    (x.val, y.val), label.val = choice(dataset)
    f.compute()
    f.backprop()

    a.grad -= a.val * regularization
    b.grad -= b.val * regularization

    f.update_parameters(0.1)

The problem is that the `regularization` applied to parameters `a` and `b` dominates the parameter update.

Making `regularization` very small or 0 allows for convergence, although at an estremely slow pace.

In [ ]:
a, b, c = param(1, -2, -1)  # initial solution
x, y, label = const(0, 0, 0)  # not affected by backprop
f = minimum(1, label * (a * x + b * y + c))

for iteration in range(35001):

    if iteration % 2500 == 0 or (iteration % 10 == 0 and iteration < 40):
        correct = sum(f.compute() > 0 for (x.val, y.val), label.val in dataset)
        print 'Accuracy at iteration {}: {:.1f} [{:.2f} {:.2f} {:.2f}]'.format(
            iteration, (100.0 * correct) / len(dataset), a.val, b.val, c.val)

    (x.val, y.val), label.val = choice(dataset)
    f.compute()
    f.backprop(0.1)

Using a single sample to compute gradients produces an update direction nearly orthogonal to that of the closest optimal solution.

Using a batch size larger than 1 or the full dataset does not help, since the projection of Jacobian in the direction of the closest optimal solution is regardless very small (proportional to the amount of gradients computed).

Adding a global multiplier to the `f` function results in a 3X speed-up despite adding a degree of freedom:

In [ ]:
a, b, c, m = param(1, -2, -1, 1)  # initial solution
x, y, label = const(0, 0, 0)  # not affected by backprop
f = minimum(1, label * m * (a * x + b * y + c))

for iteration in range(12001):

    if iteration % 500 == 0 or (iteration % 10 == 0 and iteration < 40):
        correct = sum(f.compute() > 0 for (x.val, y.val), label.val in dataset)
        print 'Accuracy at iteration {}: {:.1f} [{:.2f} * ({:.2f}, {:.2f}, {:.2f})]'.format(
            iteration, (100.0 * correct) / len(dataset), m.val, a.val, b.val, c.val)

    (x.val, y.val), label.val = choice(dataset)
    f.compute()
    f.backprop(0.1)

Asking for the norm of `(a, b, c)` to be 1 again creates competition between the two objectives, making the constraint either counterproductive or ineffective. Making it converge requires careful balancing of gradients propagated and the learning rate.

In [1]:
a, b, c, m = param(1, -2, -1, 1)  # initial solution
x, y, label = const(0, 0, 0)  # not affected by backprop
f = minimum(1, label * m * (a * x + b * y + c))
k = norm(a, b, c)  # konstraint

for iteration in range(60000):

    if iteration % 500 == 0 or (iteration % 10 == 0 and iteration < 40):
        correct = sum(f.compute() > 0 for (x.val, y.val), label.val in dataset)
        print 'Accuracy at iteration {}: {:.1f} [{:.2f} {:.2f} {:.2f} * {:.2f}]'.format(
            iteration, (100.0 * correct) / len(dataset), a.val, b.val, c.val, m.val)

    (x.val, y.val), label.val = choice(dataset)

    f.compute()
    k.compute()  # this zeros gradients on a, b, c

    f.backprop(grad=1)
    k.backprop(grad=0.01 * (1 if k < 1 else -1))

    f.update_parameters(lr=0.01)

Accuracy at iteration 0: 66.7 [1.00 -2.00 -1.00]
Accuracy at iteration 10: 66.7 [0.76 -2.00 -0.70]
Accuracy at iteration 20: 83.3 [0.22 -2.09 -0.10]
Accuracy at iteration 30: 83.3 [0.16 -2.06 -0.00]


Accuracy at iteration 2500: 83.3 [0.90 -4.97 1.90]


Accuracy at iteration 5000: 83.3 [1.29 -7.29 2.90]


Accuracy at iteration 7500: 83.3 [1.86 -9.54 4.40]


Accuracy at iteration 10000: 83.3 [2.25 -11.78 5.40]


...

Accuracy at iteration 25000: 100.0 [4.35 -24.37 12.50]


Accuracy at iteration 27500: 100.0 [4.68 -26.08 13.40]


Accuracy at iteration 30000: 100.0 [4.92 -27.60 14.20]


Accuracy at iteration 32500: 100.0 [5.04 -27.78 14.40]


Accuracy at iteration 35000: 100.0 [5.04 -27.78 14.40]


In [3]:
from random import choice
from utils.sugar import *

dataset = (((1.2, 0.7), +1.0), ((-0.3, 0.5), -1.0), ((-3.0, -1.0), +1.0),
           ((0.1, 1.0), -1.0), ((3.0, 1.1), -1.0), ((2.1, -3.0), +1.0))

a, b, c = param(1, -2, -1)  # initial solution
x, y, label = const(0, 0, 0)  # not affected by backprop
f = minimum(1, label * (a * x + b * y + c))

for iteration in range(35001):

    if iteration % 2500 == 0 or (iteration % 10 == 0 and iteration < 40):
        correct = sum(f.compute() > 0 for (x.val, y.val), label.val in dataset)
        print 'Accuracy at iteration {}: {:.1f} [{:.2f} {:.2f} {:.2f}]'.format(
            iteration, (100.0 * correct) / len(dataset), a.val, b.val, c.val)

    (x.val, y.val), label.val = choice(dataset)
    f.compute()
    f.backprop()
    
    # a.grad += -a.val
    # b.grad += -b.val

    f.update_parameters(0.1)

Accuracy at iteration 0: 66.7 [1.00 -2.00 -1.00]
Accuracy at iteration 10: 83.3 [0.28 -2.13 -0.40]
Accuracy at iteration 20: 83.3 [0.22 -2.09 -0.10]


Accuracy at iteration 30: 83.3 [0.34 -2.02 -0.00]


Accuracy at iteration 2500: 83.3 [0.94 -4.96 1.80]


Accuracy at iteration 5000: 83.3 [1.69 -7.12 3.10]


Accuracy at iteration 7500: 83.3 [1.57 -9.47 4.20]


...

Accuracy at iteration 25000: 100.0 [4.45 -24.39 12.60]


Accuracy at iteration 27500: 100.0 [4.78 -26.19 13.60]


Accuracy at iteration 30000: 100.0 [5.11 -27.80 14.30]


Accuracy at iteration 32500: 100.0 [5.11 -28.15 14.60]


Accuracy at iteration 35000: 100.0 [5.11 -28.15 14.60]


Javascript code
```javascript
var n1 = Math.max(0, a1*x + b1*y + c1); // activation of 1st hidden neuron
var n2 = Math.max(0, a2*x + b2*y + c2); // 2nd neuron
var n3 = Math.max(0, a3*x + b3*y + c3); // 3rd neuron
var score = a4*n1 + b4*n2 + c4*n3 + d4;
```

In [ ]:
from random import random
from utils.sugar import *

dataset = (((1.2, 0.7), +1.0), ((-0.3, 0.5), -1.0), ((-3.0, -1.0), +1.0),
           ((0.1, 1.0), -1.0), ((3.0, 1.1), -1.0), ((2.1, -3.0), +1.0))


def hn(xx, yy):  # hidden neuron relu(ax + by + c)
    return relu(param() * xx + param() * yy + param())


x, y = const(0, 0)  # score = w_0 + sum_1^3 w_i * hn_i(x, y)
score = summation([param()] + [param() * hn(x, y) for _ in range(3)])

for par in score.parameters():  # randomly initialize weights
    par.val = random() - 0.5

for iteration in range(35001):

    if iteration % 2500 == 0 or (iteration % 10 == 0 and iteration < 40):
        correct = sum(score.compute() > 0 for (x.val, y.val), label.val in dataset)
        print 'Accuracy at iteration {}: {:.1f} [{:.2f} {:.2f} {:.2f}]'.format(
            iteration, (100.0 * correct) / len(dataset), a.val, b.val, c.val)

    (x.val, y.val), label.val = choice(dataset)
    score.compute()
    score.backprop(0.01)
